In [1]:
from sqlalchemy import create_engine
from cubes.tutorial.sql import create_table_from_csv
from cubes import Workspace

In [2]:
engine = create_engine('sqlite:///data.sqlite')
create_table_from_csv(engine,
                      "IBRD_Balance_Sheet__FY2010.csv",
                      table_name="ibrd_balance",
                      fields=[
                          ("category", "string"),
                          ("category_label", "string"),
                          ("subcategory", "string"),
                          ("subcategory_label", "string"),
                          ("line_item", "string"),
                          ("year", "integer"),
                          ("amount", "integer")],
                      create_id=True
                     )

workspace = Workspace()
workspace.register_default_store("sql", url="sqlite:///data.sqlite")
workspace.import_model("tutorial_model.json")
cube = workspace.cube("ibrd_balance")
browser = workspace.browser(cube)

In [3]:
result = browser.aggregate()
result.summary["record_count"]

62

In [4]:
result.summary["total_amount"]

1116860

In [5]:
result = browser.aggregate(drilldown=["item"])
for record in result:
    print(record)

{'item.category': 'a', 'item.category_label': 'Assets', 'total_amount': 558430, 'maximum_amount': 118104, 'minimum_amount': 0, 'record_count': 32}
{'item.category': 'e', 'item.category_label': 'Equity', 'total_amount': 77592, 'maximum_amount': 29870, 'minimum_amount': -3043, 'record_count': 8}
{'item.category': 'l', 'item.category_label': 'Liabilities', 'total_amount': 480838, 'maximum_amount': 128577, 'minimum_amount': 8, 'record_count': 22}


In [6]:
import cubes as cubes
cuts = [cubes.PointCut("year", ["2009"])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell, drilldown=["item"])
for record in result:
    print(record)

{'item.category': 'a', 'item.category_label': 'Assets', 'total_amount': 275420, 'maximum_amount': 103657, 'minimum_amount': 33, 'record_count': 16}
{'item.category': 'e', 'item.category_label': 'Equity', 'total_amount': 40037, 'maximum_amount': 29870, 'minimum_amount': -1683, 'record_count': 4}
{'item.category': 'l', 'item.category_label': 'Liabilities', 'total_amount': 235383, 'maximum_amount': 110040, 'minimum_amount': 57, 'record_count': 11}


In [7]:
cuts = [cubes.PointCut("year", ["2009"]),cubes.PointCut("item", ["a"])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell,drilldown=["item"])
result.summary

{'total_amount': 275420,
 'maximum_amount': 103657,
 'minimum_amount': 33,
 'record_count': 16}

In [8]:
cuts = [cubes.PointCut("year", ["2009"]),cubes.PointCut("item", ["a"])]
cell = cubes.Cell(cube, cuts)
result = browser.aggregate(cell,drilldown=["item:subcategory"])
for record in result:
    print(record)

{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'da', 'item.subcategory_label': 'Derivative Assets', 'total_amount': 123065, 'maximum_amount': 82793, 'minimum_amount': 2246, 'record_count': 4}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'dfb', 'item.subcategory_label': 'Due from Banks', 'total_amount': 3044, 'maximum_amount': 2380, 'minimum_amount': 664, 'record_count': 2}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'i', 'item.subcategory_label': 'Investments', 'total_amount': 41012, 'maximum_amount': 41012, 'minimum_amount': 41012, 'record_count': 1}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'lo', 'item.subcategory_label': 'Loans Outstanding', 'total_amount': 103657, 'maximum_amount': 103657, 'minimum_amount': 103657, 'record_count': 1}
{'item.category': 'a', 'item.category_label': 'Assets', 'item.subcategory': 'nn', 'item.subcategory_label': 'Nonnegotiable', '

## <font color = 'maroon'>Assignment Part 01</font> {-}

### <font color = 'maroon'> 1. Music Streaming Company </font> {-}

#### A. Schema Diagram: 

Designed using snowflake Schema: answer attached at the bottom of Part 01 as it was not created via notebook

#### B. OLAP operations to be performed:

In order to list the total fee collected for a given song for a given month of a given year (e.g. October 2020)

Time dimension conceptual hierarchy: day < day of week < month < quarter < year

1. Roll up from "day" to "month" for time dimension
2. Roll up to "all" for user and song dimensions
3. Dice for specific song, month and year
4. Perform the sum() operation on the total_fee

#### C. Number of Cuboids: 

The number of cuboids is given by the formula
$T$=$\Pi_{i=1}^n (L_{i} + 1)$, 

where $L_{i}$ is the number of levels associated with the dimension i.
One is added to include the virtual top level, all.

Here, L = 5, 1, 1 respectively for i = 1,2,3 

Therefore substituting on formula, 

(5+1)(1+1)(1+1) = 6x2x2 = 24 cuboids

### <font color = 'maroon'> 2. Regional Rainfall </font> {-}

The data cube has dimensions region, precipitation, and time

#### A. Data measure to be used to compute the total amount of rainfall for a given region and month


**data cube measure:** sum() is applied on the dimension **precipitation** rolled up to region and time (month)

**category:** distributive

**explanation:** because this function can be performed in a distrubutive manner and the final output is not affected or changed.


#### B. Data measure to be used to compute the average rainfall for a given region and month

**data cube measure:** avg() applied on the dimension **precipitation** rolled up to region and time (month)

**category:** algebraic

**explanation:** because this function performs algebraic aggregate functions, ie., $average = \frac {sum}{count}$ and sum() and count() are distributive aggregate functions

### <font color = 'maroon'> 3. Indexing OLAP data Using bitmap indices </font> {-}

**Base Table**

| RID   | brand   | branch         |                    
|:-|:-|:-|
|   R1  | Audi    | Tower Hamlets  |
|   R2  | Audi    | Newham         |
|   R3  | Audi    | Hackney        |
|   R4  | Ford    | Tower Hamlets  |
|   R5  | Ford    | Newham         |
|   R6  | Ford    | Hackney        |
|   R7  | Mini    | Tower Hamlets  |
|   R8  | Mini    | Newham         |
|   R9  | Mini    | Hackney        |

*vehicle brand* **Bitmap Index Table**

| RID   | Audi | Ford | Mini |                  
|:-:|:-:|:-:|:-:|
|   R1  | 1 | 0 | 0 |
|   R2  | 1 | 0 | 0 |
|   R3  | 1 | 0 | 0 |
|   R4  | 0 | 1 | 0 | 
|   R5  | 0 | 1 | 0 | 
|   R6  | 0 | 1 | 0 | 
|   R7  | 0 | 0 | 1 | 
|   R8  | 0 | 0 | 1 | 
|   R9  | 0 | 0 | 1 | 

*store branch* **Bitmap Index Table**

| RID   | Tower Hamlets | Newham | Hackney |                  
|:-:|:-:|:-:|:-:|
|   R1  | 1 | 0 | 0 |
|   R2  | 0 | 1 | 0 |
|   R3  | 0 | 0 | 1 |
|   R4  | 1 | 0 | 0 | 
|   R5  | 0 | 1 | 0 | 
|   R6  | 0 | 0 | 1 | 
|   R7  | 1 | 0 | 0 | 
|   R8  | 0 | 1 | 0 | 
|   R9  | 0 | 0 | 1 | 

### <font color = 'maroon'> 4. modifying the "tutorial_model.json" file to include aggregate measures for the minimum and maximum amount in the data cube and producing the values for the minimum and maximum amount in the data per year</font> {-}

json updated to include the aggregate functions min and max on the amount measure. 

The code is pasted below for reference and the succeeding cell displays the updated results.

```json
{
    "dimensions": [
        {
         "name":"item",
         "levels": [
                {
                    "name":"category",
                    "label":"Category",
                    "attributes": ["category", "category_label"]
                },
                {
                    "name":"subcategory",
                    "label":"Sub-category",
                    "attributes": ["subcategory", "subcategory_label"]
                },
                {
                    "name":"line_item",
                    "label":"Line Item",
                    "attributes": ["line_item"]
                }
            ]
        },
        {"name":"year", "role": "time"}
    ],
    "cubes": [
        {
            "name": "ibrd_balance",
            "dimensions": ["item", "year"],
            "measures": [{"name":"amount", "label":"Amount"}],
            "aggregates": [
                    {
                        "name": "total_amount",
                        "function": "sum",
                        "measure": "amount"
                    },
                    {
                        "name": "maximum_amount",
                        "function": "max",
                        "measure": "amount"
                    }, 
                    {
                        "name": "minimum_amount",
                        "function": "min",
                        "measure": "amount"
                    },                  
                    {
                        "name": "record_count",
                        "function": "count"
                    }
                ],
            "mappings": {
                          "item.line_item": "line_item",
                          "item.subcategory": "subcategory",
                          "item.subcategory_label": "subcategory_label",
                          "item.category": "category",
                          "item.category_label": "category_label"
                         },
            "info": {
                "min_date": "2010-01-01",
                "max_date": "2010-12-31"
            }
        }
    ]
}
```

In [9]:
result = browser.aggregate(drilldown=["year"])
for record in result:
    print(record)

{'year': 2009, 'total_amount': 550840, 'maximum_amount': 110040, 'minimum_amount': -1683, 'record_count': 31}
{'year': 2010, 'total_amount': 566020, 'maximum_amount': 128577, 'minimum_amount': -3043, 'record_count': 31}


### <font color = 'maroon'> 5. Performing the following on  "country-income.csv":</font> {-}

####  1. Load the CSV file using Cubes, create a JSON file for the data cube model, and create a data cube for the data. Use as dimensions the region, age, and online shopper fields. Use as measure the income. Define aggregate functions in the data cube model for the total, average, minimum, and maximum income. 

In [10]:
engine = create_engine('sqlite:///data.sqlite')
create_table_from_csv(engine,
                      "country-income.csv",
                      table_name="country_income",
                      fields=[
                          ("region", "string"),
                          ("age", "integer"),
                          ("income", "integer"),
                          ("online", "string")
                      ],
                      create_id=True
                     )

#### "country_income_model.json"

```json
{
    "dimensions": [
        {
            "name":"region", "label": "Region"
        },
        {
            "name":"age", "label": "Age"
        },
        {
            "name":"online", "label": "Online_shopper"
        }
    ],
    "cubes": [
        {
            "name": "country_income",
            "dimensions": ["region","age","online"],
            "measures": [{"name":"income", "label":"Income"}],
            "aggregates": [
                    {
                        "name": "total_income",
                        "function": "sum",
                        "measure": "income"
                    },       
                    {
                        "name": "minimum_income",
                        "function": "min",
                        "measure": "income"
                    },   
                    {
                        "name": "maximum_income",
                        "function": "max",
                        "measure": "income"
                    },                 
                    {
                        "name": "average_income",
                        "function": "avg",
                        "measure": "income"
                    }
                ]
        }
    ]
}

```

In [11]:
workspace = Workspace()
workspace.register_default_store("sql", url="sqlite:///data.sqlite")
workspace.import_model("country_income_model.json")
cube = workspace.cube("country_income")
browser = workspace.browser(cube)

#### 2. aggregate:

##### RESULTS FOR THE WHOLE DATA CUBE:

In [12]:
result1 = browser.aggregate()
result1.summary

{'total_income': 768200,
 'minimum_income': 57600,
 'maximum_income': 99600,
 'average_income': 76820.0}

##### RESULTS PER REGION:

In [13]:
result2 = browser.aggregate(drilldown=["region"])
for record in result2:
    print(record)

{'region': 'Brazil', 'total_income': 193200, 'minimum_income': 57600, 'maximum_income': 73200, 'average_income': 64400.0}
{'region': 'India', 'total_income': 331200, 'minimum_income': 69600, 'maximum_income': 94800, 'average_income': 82800.0}
{'region': 'USA', 'total_income': 243800, 'minimum_income': 64800, 'maximum_income': 99600, 'average_income': 81266.66666666667}


##### RESULTS PER ONLINE SHOPPING ACTIVITY:

In [14]:
result3 = browser.aggregate(drilldown=["online"])
for record in result3:
    print(record)

{'online': 'No', 'total_income': 386400, 'minimum_income': 62400, 'maximum_income': 99600, 'average_income': 77280.0}
{'online': 'Yes', 'total_income': 381800, 'minimum_income': 57600, 'maximum_income': 94800, 'average_income': 76360.0}


##### RESULTS FOR ALL PEOPLE AGED BETWEEN 40 AND 50: 

In [15]:
cuts = [cubes.RangeCut("age", [40],[50])]
cell = cubes.Cell(cube, cuts)
print(cell)
result4 = browser.aggregate(cell, drilldown=["age"])
result4.summary

age:40-50


{'total_income': 451400,
 'minimum_income': 62400,
 'maximum_income': 86400,
 'average_income': 75233.33333333333}

In [16]:
cuts = [cubes.RangeCut("age", [40],[50])]
cell = cubes.Cell(cube, cuts)
result5 = browser.aggregate(cell, drilldown=["age"])
for record in result5:
    print(record)

{'age': 40, 'total_income': 69600, 'minimum_income': 69600, 'maximum_income': 69600, 'average_income': 69600.0}
{'age': 42, 'total_income': 80400, 'minimum_income': 80400, 'maximum_income': 80400, 'average_income': 80400.0}
{'age': 43, 'total_income': 73200, 'minimum_income': 73200, 'maximum_income': 73200, 'average_income': 73200.0}
{'age': 45, 'total_income': 79400, 'minimum_income': 79400, 'maximum_income': 79400, 'average_income': 79400.0}
{'age': 46, 'total_income': 62400, 'minimum_income': 62400, 'maximum_income': 62400, 'average_income': 62400.0}
{'age': 49, 'total_income': 86400, 'minimum_income': 86400, 'maximum_income': 86400, 'average_income': 86400.0}
